In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
df_train = pd.read_csv('data_vasochange/train_withterm.csv')
df_train.head()

,bloc,icustayid,mortality_90d,MIMICzs1,MIMICzs2,MIMICzs3,MIMICzs4,MIMICzs5,MIMICzs6,MIMICzs7,...,MIMICzs44,MIMICzs45,MIMICzs46,MIMICzs47,action,shaped_reward,io_ac,vc_ac,abchange_vc,max_dose_vaso
0,1,3,1,-0.5,-0.5,-0.5,-2.302585,-0.978344,-0.187300,0.705956,...,0.391651,0.52761,0.786192,0.702781,10,-1.224270,2,0,0.0,0.0
1,2,3,1,-0.5,-0.5,-0.5,-2.302585,-0.978344,-0.187300,0.705956,...,0.394176,0.52761,0.793676,0.596530,10,1.640796,2,0,0.0,0.0
2,3,3,1,-0.5,-0.5,-0.5,-2.302585,-0.978344,-0.187300,0.705956,...,0.396682,0.52761,0.799286,0.516950,10,-0.025000,2,0,0.0,0.0
3,4,3,1,-0.5,-0.5,-0.5,-2.302585,-0.978344,-0.155313,0.705956,...,0.399169,0.52761,0.805952,0.575231,10,-0.025000,2,0,0.0,0.0
4,5,3,1,-0.5,-0.5,-0.5,-2.302585,-0.978344,-0.147317,0.705956,...,0.401637,0.52761,0.816227,0.714111,10,-0.025000,2,0,0.0,0.0


In [3]:
df_val = pd.read_csv('data_vasochange/val_withterm.csv')

In [4]:
df_test = pd.read_csv('data_vasochange/test_withterm.csv')

In [5]:
agent_reward_estimates_val = pickle.load(open('./val_rewards.p', "rb" ))
agent_reward_estimates_test = pickle.load(open('./test_rewards.p', "rb" ))
#agent_reward_estimates_train = pickle.load(open('./train_rewards.p', "rb" ))
# agent_reward_estimates = agent_q

In [6]:
df_val['reward_estimate'] = agent_reward_estimates_val
df_test['reward_estimate'] = agent_reward_estimates_test
#df_train['reward_estimate'] = agent_reward_estimates_train

In [7]:
action_map = {}
count = 0
for iv in range(5):
    for vaso in range(5):
        action_map[(iv,vaso)] = count
        count += 1

In [8]:
print(action_map)

{(0, 0): 0, (0, 1): 1, (0, 2): 2, (0, 3): 3, (0, 4): 4, (1, 0): 5, (1, 1): 6, (1, 2): 7, (1, 3): 8, (1, 4): 9, (2, 0): 10, (2, 1): 11, (2, 2): 12, (2, 3): 13, (2, 4): 14, (3, 0): 15, (3, 1): 16, (3, 2): 17, (3, 3): 18, (3, 4): 19, (4, 0): 20, (4, 1): 21, (4, 2): 22, (4, 3): 23, (4, 4): 24}


In [9]:
unique_ids_val = df_val['icustayid'].unique()
unique_ids_test = df_test['icustayid'].unique()
unique_ids_train = df_train['icustayid'].unique()

In [10]:
phys_vals = []
gamma = 0.99
for uid in unique_ids_test:
    traj = df_test.loc[df_test['icustayid'] == uid]
    ret = 0
    reversed_traj = traj.iloc[::-1]
    for row in reversed_traj.index:
        ret = reversed_traj.loc[row,'shaped_reward'] + gamma*ret
    phys_vals.append(ret)
print(np.mean(phys_vals))
print(max(phys_vals))
print(min(phys_vals))

7.1651775867517244
23.16690776805195
-23.148286743330715


In [11]:
phys_vals = []
gamma = 0.99
for uid in unique_ids_test:
    traj = df_test.loc[df_test['icustayid'] == uid]
    ret = 0
    reversed_traj = traj.iloc[::-1]
    for row in reversed_traj.index:
        ret = reversed_traj.loc[row,'reward_estimate'] + gamma*ret
    phys_vals.append(ret)
print(np.mean(phys_vals))
print(max(phys_vals))
print(min(phys_vals))

9.791838923054046
37.77802560247752
-31.008447960683625
